In [1]:
# for database connections
from sqlalchemy import create_engine, inspect, text

# data
import pandas as pd

# Postgres credentials
import config as cfg

In [2]:
# Setup the SQL connection variables
SQL_USERNAME = cfg.SQL_USERNAME
SQL_PASSWORD = cfg.SQL_PASSWORD
SQL_IP = cfg.SQL_IP
SQL_PORT = cfg.SQL_PORT
DATABASE = cfg.DATABASE

In [3]:
# Connect to PostgreSQL server
connection_string = f'postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{SQL_PORT}/{DATABASE}'
engine = create_engine(connection_string)

## upload campaign, category, subcategory and contact tables

In [6]:
# Check to see if the category and subcategory tables are in the database
inspector = inspect(engine)
print(inspector.get_table_names())

['income', 'jobs', 'unemployment', 'employment']


In [7]:
# Display all the columns of the tables in PostgreSQL to make sure they have the correct columns prior to loading the data

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within each table and its types
for table in tables:
    print(f"Table name: {table}")
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])
        
    print()

Table name: income
fips VARCHAR(5)
state VARCHAR(2)
county VARCHAR(40)
median_hh_inc_acs INTEGER
percapita_inc INTEGER
poverty_rate_0_17_acs NUMERIC
poverty_rate_acs NUMERIC
deep_pov_all NUMERIC
deep_pov_children NUMERIC
num_allinpov_acs INTEGER
pct_pov_0_17 NUMERIC
pov_0_17 INTEGER
med_hh_inc INTEGER
pov_all INTEGER
pct_pov_all NUMERIC
num_in_pov_0_17_acs INTEGER
last_update TIMESTAMP

Table name: jobs
fips VARCHAR(5)
state VARCHAR(2)
county VARCHAR(40)
pctemp_agriculture NUMERIC
pctemp_mining NUMERIC
pctemp_construction NUMERIC
pctemp_manufacturing NUMERIC
pctemp_trade NUMERIC
pctemp_trans NUMERIC
pctemp_information NUMERIC
pctemp_fire NUMERIC
pctemp_services NUMERIC
pctemp_government NUMERIC
num_civ_employed INTEGER
last_update TIMESTAMP

Table name: unemployment
fips VARCHAR(5)
year VARCHAR(4)
unemp_rate NUMERIC
num_unemployed INTEGER
last_update TIMESTAMP

Table name: employment
fips VARCHAR(5)
year VARCHAR(4)
num_civ_labor_force INTEGER
num_employed INTEGER
last_update TIMESTAMP


In [ ]:
# Open category csv file and read it into a pandas dataframe
income_df = pd.read_csv('DB_Input/income.csv')
income_df.head()

In [ ]:
# Open subcategory csv file and read it into a pandas dataframe
employment_df = pd.read_csv('DB_Input/employment.csv')
employment_df.head()

In [ ]:
# Open contact csv file and read it into a pandas dataframe
unemployment_df = pd.read_csv('DB_Input/unemployment.csv')
unemployment_df.head()

In [ ]:
# Open campaign csv file and read it into a pandas dataframe
jobs_df = pd.read_csv('DB_Input/jobs.csv')
jobs_df.head()

In [ ]:
# If category table exists in the database, load the category data into the table
if 'income' in inspector.get_table_names():
    income_df.to_sql('income', schema='public', con=engine, index=False, if_exists='append', method='multi')

In [ ]:
# If subcategory table exists in the database, load the subcategory data into the table
if 'employment' in inspector.get_table_names():
    subcategory_df.to_sql('employment', schema='public', con=engine, index=False, if_exists='append', method='multi')

In [ ]:
# If contact table exists in the database, load the contact data into the table
if 'unemployment' in inspector.get_table_names():
    contact_df.to_sql('unemployment', schema='public', con=engine, index=False, if_exists='append', method='multi')

In [ ]:
# If campaign table exists in the database, load the campaign data into the table
if 'jobs' in inspector.get_table_names():
    campaign_df.to_sql('jobs', schema='public', con=engine, index=False, if_exists='append', method='multi')